# 01. 로컬 환경 Northwind 데이터베이스 구축 (Text-to-SQL용)

이 노트북은 **로컬 SQLite 환경**에서 Text-to-SQL RAG 시스템을 위한 Northwind 샘플 데이터베이스를 구축합니다.

## 🎯 목표
- **SQLite 데이터베이스** 생성 및 연결
- **Northwind 샘플 데이터베이스** 스키마 설계 및 구축
- **샘플 데이터** 삽입 및 검증
- **Text-to-SQL 시스템** 연동 준비

## 🏗️ Northwind 데이터베이스 구조
```
Northwind Database (8개 테이블)
├── customers      # 고객 정보 (91개)
├── suppliers      # 공급업체 정보 (29개)
├── categories     # 상품 카테고리 (8개)
├── products       # 상품 정보 (77개)
├── employees      # 직원 정보 (9개)
├── shippers       # 운송업체 정보 (3개)
├── orders         # 주문 정보 (830개)
└── order_details  # 주문 상세 정보 (2155개)
```

## 📋 전제조건
- Python 3.7+
- SQLite3 (Python 내장)
- pandas, numpy

## 🚀 다음 단계
이 노트북 완료 후 → `02_langchain_agent_text_to_sql.ipynb`에서 LangChain Agent 구현

## 🗄️ Northwind 샘플 데이터베이스 소개

Northwind는 **가상의 무역회사 데이터베이스**로 Text-to-SQL 학습에 이상적입니다.

### 📊 테이블별 상세 정보
- **Categories** (8개): 음료, 조미료, 유제품, 해산물, 곡물/시리얼, 육류, 농산물, 과자
- **Suppliers** (29개): 전 세계 다양한 공급업체 정보
- **Products** (77개): 각 카테고리별 다양한 상품
- **Customers** (91개): 전 세계 고객사 정보
- **Employees** (9개): 직원 정보 및 상하관계
- **Shippers** (3개): Speedy Express, United Package, Federal Shipping
- **Orders** (830개): 1996-1998년 주문 데이터
- **Order Details** (2155개): 상품별 주문 상세 정보

### 🎯 Text-to-SQL에 적합한 이유
1. **실무 친화적**: 실제 비즈니스 시나리오 반영
2. **다양한 관계**: JOIN, 집계, 필터링 등 다양한 SQL 패턴
3. **직관적 데이터**: 도메인 지식이 쉽게 이해 가능
4. **적당한 크기**: 학습 및 테스트에 최적화된 데이터 규모

In [ ]:
# 1. 필요한 라이브러리 임포트 및 환경 설정
import sqlite3
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime, date
import warnings
warnings.filterwarnings('ignore')

print("🚀 로컬 Northwind Text-to-SQL 데이터베이스 구축 시작")
print("=" * 60)

# 환경 정보 확인
print(f"📋 환경 정보:")
print(f"   Python: {pd.__version__} (pandas)")
print(f"   NumPy: {np.__version__}")
print(f"   작업 디렉토리: {Path.cwd()}")

# 데이터 디렉토리 생성
data_dir = Path("data")
data_dir.mkdir(exist_ok=True)

print(f"📂 데이터 디렉토리: {data_dir.absolute()}")
print(f"📅 시작 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# 2. SQLite 데이터베이스 연결 및 초기화
db_path = data_dir / "northwind.db"

print(f"🔗 SQLite 데이터베이스 연결...")
print(f"📂 데이터베이스 경로: {db_path.absolute()}")

try:
    # 기존 데이터베이스 파일이 있다면 백업
    if db_path.exists():
        backup_path = db_path.with_suffix('.backup.db')
        db_path.rename(backup_path)
        print(f"💾 기존 데이터베이스 백업: {backup_path.name}")
    
    # 새로운 데이터베이스 연결
    conn = sqlite3.connect(str(db_path))
    cursor = conn.cursor()
    
    # SQLite 설정 최적화
    cursor.execute("PRAGMA foreign_keys = ON")  # 외래키 제약조건 활성화
    cursor.execute("PRAGMA journal_mode = WAL")  # 성능 최적화
    
    print("✅ SQLite 데이터베이스 연결 성공!")
    
    # SQLite 버전 확인
    cursor.execute("SELECT sqlite_version()")
    sqlite_version = cursor.fetchone()[0]
    print(f"🔗 SQLite 버전: {sqlite_version}")
    
    # 설정 확인
    cursor.execute("PRAGMA foreign_keys")
    fk_status = cursor.fetchone()[0]
    print(f"🔐 외래키 제약조건: {'✅ 활성화' if fk_status else '❌ 비활성화'}")
    
except Exception as e:
    print(f"❌ 데이터베이스 연결 실패: {str(e)}")
    raise

In [ ]:
# 3. Northwind 데이터베이스 스키마 생성
print("🏗️ Northwind 데이터베이스 스키마 생성...")

# 테이블 생성 순서 (외래키 의존성 고려)
schema_sql = {
    "categories": """
        CREATE TABLE categories (
            category_id INTEGER PRIMARY KEY,
            category_name VARCHAR(15) NOT NULL,
            description TEXT,
            picture BLOB
        )
    """,
    
    "suppliers": """
        CREATE TABLE suppliers (
            supplier_id INTEGER PRIMARY KEY,
            company_name VARCHAR(40) NOT NULL,
            contact_name VARCHAR(30),
            contact_title VARCHAR(30),
            address VARCHAR(60),
            city VARCHAR(15),
            region VARCHAR(15),
            postal_code VARCHAR(10),
            country VARCHAR(15),
            phone VARCHAR(24),
            fax VARCHAR(24),
            home_page TEXT
        )
    """,
    
    "products": """
        CREATE TABLE products (
            product_id INTEGER PRIMARY KEY,
            product_name VARCHAR(40) NOT NULL,
            supplier_id INTEGER,
            category_id INTEGER,
            quantity_per_unit VARCHAR(20),
            unit_price DECIMAL(10,4) DEFAULT 0,
            units_in_stock SMALLINT DEFAULT 0,
            units_on_order SMALLINT DEFAULT 0,
            reorder_level SMALLINT DEFAULT 0,
            discontinued INTEGER NOT NULL DEFAULT 0,
            FOREIGN KEY (supplier_id) REFERENCES suppliers(supplier_id),
            FOREIGN KEY (category_id) REFERENCES categories(category_id)
        )
    """,
    
    "customers": """
        CREATE TABLE customers (
            customer_id VARCHAR(5) PRIMARY KEY,
            company_name VARCHAR(40) NOT NULL,
            contact_name VARCHAR(30),
            contact_title VARCHAR(30),
            address VARCHAR(60),
            city VARCHAR(15),
            region VARCHAR(15),
            postal_code VARCHAR(10),
            country VARCHAR(15),
            phone VARCHAR(24),
            fax VARCHAR(24)
        )
    """,
    
    "employees": """
        CREATE TABLE employees (
            employee_id INTEGER PRIMARY KEY,
            last_name VARCHAR(20) NOT NULL,
            first_name VARCHAR(10) NOT NULL,
            title VARCHAR(30),
            title_of_courtesy VARCHAR(25),
            birth_date DATE,
            hire_date DATE,
            address VARCHAR(60),
            city VARCHAR(15),
            region VARCHAR(15),
            postal_code VARCHAR(10),
            country VARCHAR(15),
            home_phone VARCHAR(24),
            extension VARCHAR(4),
            photo BLOB,
            notes TEXT,
            reports_to INTEGER,
            photo_path VARCHAR(255),
            FOREIGN KEY (reports_to) REFERENCES employees(employee_id)
        )
    """,
    
    "shippers": """
        CREATE TABLE shippers (
            shipper_id INTEGER PRIMARY KEY,
            company_name VARCHAR(40) NOT NULL,
            phone VARCHAR(24)
        )
    """,
    
    "orders": """
        CREATE TABLE orders (
            order_id INTEGER PRIMARY KEY,
            customer_id VARCHAR(5),
            employee_id INTEGER,
            order_date DATE,
            required_date DATE,
            shipped_date DATE,
            ship_via INTEGER,
            freight DECIMAL(10,4) DEFAULT 0,
            ship_name VARCHAR(40),
            ship_address VARCHAR(60),
            ship_city VARCHAR(15),
            ship_region VARCHAR(15),
            ship_postal_code VARCHAR(10),
            ship_country VARCHAR(15),
            FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
            FOREIGN KEY (employee_id) REFERENCES employees(employee_id),
            FOREIGN KEY (ship_via) REFERENCES shippers(shipper_id)
        )
    """,
    
    "order_details": """
        CREATE TABLE order_details (
            order_id INTEGER,
            product_id INTEGER,
            unit_price DECIMAL(10,4) NOT NULL,
            quantity SMALLINT NOT NULL,
            discount REAL NOT NULL DEFAULT 0,
            PRIMARY KEY (order_id, product_id),
            FOREIGN KEY (order_id) REFERENCES orders(order_id),
            FOREIGN KEY (product_id) REFERENCES products(product_id)
        )
    """
}

# 스키마 실행
try:
    print(f"📊 총 {len(schema_sql)}개 테이블 생성 중...")
    
    for table_name, sql in schema_sql.items():
        cursor.execute(sql)
        print(f"   ✅ {table_name} 테이블 생성 완료")
    
    conn.commit()
    print("\n🎉 Northwind 스키마 생성 완료!")
    
    # 생성된 테이블 확인
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = [row[0] for row in cursor.fetchall()]
    
    print(f"\n📋 생성된 테이블 ({len(tables)}개):")
    for table in tables:
        print(f"   📂 {table}")
    
except Exception as e:
    print(f"❌ 스키마 생성 실패: {str(e)}")
    conn.rollback()
    raise

In [ ]:
# 4. 샘플 데이터 삽입 - Categories (카테고리)
print("📦 카테고리 데이터 삽입...")

categories_data = [
    (1, 'Beverages', 'Soft drinks, coffees, teas, beers, and ales'),
    (2, 'Condiments', 'Sweet and savory sauces, relishes, spreads, and seasonings'),
    (3, 'Dairy Products', 'Cheeses'),
    (4, 'Seafood', 'Seaweed and fish'),
    (5, 'Grains/Cereals', 'Breads, crackers, pasta, and cereal'),
    (6, 'Meat/Poultry', 'Prepared meats'),
    (7, 'Produce', 'Dried fruit and bean curd'),
    (8, 'Confections', 'Desserts, candies, and sweet breads')
]

cursor.executemany(
    "INSERT INTO categories (category_id, category_name, description) VALUES (?, ?, ?)",
    categories_data
)

print(f"✅ {len(categories_data)}개 카테고리 삽입 완료")

In [ ]:
# 5. 샘플 데이터 삽입 - Suppliers (공급업체)
print("🏢 공급업체 데이터 삽입...")

suppliers_data = [
    (1, 'Exotic Liquids', 'Charlotte Cooper', 'Purchasing Manager', '49 Gilbert St.', 'London', None, 'EC1 4SD', 'UK', '(171) 555-2222', None),
    (2, 'New Orleans Cajun Delights', 'Shelley Burke', 'Order Administrator', 'P.O. Box 78934', 'New Orleans', 'LA', '70117', 'USA', '(100) 555-4822', None),
    (3, 'Grandma Kelly\'s Homestead', 'Regina Murphy', 'Sales Representative', '707 Oxford Rd.', 'Ann Arbor', 'MI', '48104', 'USA', '(313) 555-5735', '(313) 555-3349'),
    (4, 'Tokyo Traders', 'Yoshi Nagase', 'Marketing Manager', '9-8 Sekimai Musashino-shi', 'Tokyo', None, '100', 'Japan', '(03) 3555-5011', None),
    (5, 'Cooperativa de Quesos \'Las Cabras\'', 'Antonio del Valle Saavedra', 'Export Administrator', 'Calle del Rosal 4', 'Oviedo', 'Asturias', '33007', 'Spain', '(98) 598 76 98', None),
    (6, 'Mayumi\'s', 'Mayumi Ohno', 'Marketing Representative', '92 Setsuko Chuo-ku', 'Osaka', None, '545', 'Japan', '(06) 431-7877', None),
    (7, 'Pavlova Ltd.', 'Ian Devling', 'Marketing Manager', '74 Rose St. Moonie Ponds', 'Melbourne', 'Victoria', '3058', 'Australia', '(03) 444-2343', '(03) 444-6588'),
    (8, 'Specialty Biscuits Ltd.', 'Peter Wilson', 'Sales Representative', '29 King\'s Way', 'Manchester', None, 'M14 GSD', 'UK', '(161) 555-4448', None)
]

cursor.executemany(
    """INSERT INTO suppliers (supplier_id, company_name, contact_name, contact_title, 
       address, city, region, postal_code, country, phone, fax) 
       VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""",
    suppliers_data
)

print(f"✅ {len(suppliers_data)}개 공급업체 삽입 완료")

In [ ]:
# 6. 샘플 데이터 삽입 - Products (상품)
print("🛍️ 상품 데이터 삽입...")

products_data = [
    (1, 'Chai', 1, 1, '10 boxes x 20 bags', 18.0000, 39, 0, 10, 0),
    (2, 'Chang', 1, 1, '24 - 12 oz bottles', 19.0000, 17, 40, 25, 0),
    (3, 'Aniseed Syrup', 1, 2, '12 - 550 ml bottles', 10.0000, 13, 70, 25, 0),
    (4, 'Chef Anton\'s Cajun Seasoning', 2, 2, '48 - 6 oz jars', 22.0000, 53, 0, 0, 0),
    (5, 'Chef Anton\'s Gumbo Mix', 2, 2, '36 boxes', 21.3500, 0, 0, 0, 1),
    (6, 'Grandma\'s Boysenberry Spread', 3, 2, '12 - 8 oz jars', 25.0000, 120, 0, 25, 0),
    (7, 'Uncle Bob\'s Organic Dried Pears', 3, 7, '12 - 1 lb pkgs.', 30.0000, 15, 0, 10, 0),
    (8, 'Northwoods Cranberry Sauce', 3, 2, '12 - 12 oz jars', 40.0000, 6, 0, 0, 0),
    (9, 'Mishi Kobe Niku', 4, 6, '18 - 500 g pkgs.', 97.0000, 29, 0, 0, 1),
    (10, 'Ikura', 4, 4, '12 - 200 ml jars', 31.0000, 31, 0, 0, 0),
    (11, 'Queso Cabrales', 5, 3, '1 kg pkg.', 21.0000, 22, 30, 30, 0),
    (12, 'Queso Manchego La Pastora', 5, 3, '10 - 500 g pkgs.', 38.0000, 86, 0, 0, 0),
    (13, 'Konbu', 6, 4, '2 kg box', 6.0000, 24, 0, 5, 0),
    (14, 'Tofu', 6, 7, '40 - 100 g pkgs.', 23.2500, 35, 0, 0, 0),
    (15, 'Genen Shouyu', 6, 2, '24 - 250 ml bottles', 15.5000, 39, 0, 5, 0),
    (16, 'Pavlova', 7, 8, '32 - 500 g boxes', 17.4500, 29, 0, 10, 0),
    (17, 'Alice Mutton', 7, 6, '20 - 1 kg tins', 39.0000, 0, 0, 0, 1),
    (18, 'Carnarvon Tigers', 7, 4, '16 kg pkg.', 62.5000, 42, 0, 0, 0),
    (19, 'Teatime Chocolate Biscuits', 8, 8, '10 boxes x 12 pieces', 9.2000, 25, 0, 5, 0),
    (20, 'Sir Rodney\'s Marmalade', 8, 8, '30 gift boxes', 81.0000, 40, 0, 0, 0)
]

cursor.executemany(
    """INSERT INTO products (product_id, product_name, supplier_id, category_id, 
       quantity_per_unit, unit_price, units_in_stock, units_on_order, reorder_level, discontinued) 
       VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""",
    products_data
)

print(f"✅ {len(products_data)}개 상품 삽입 완료")

In [ ]:
# 7. 샘플 데이터 삽입 - Customers (고객)
print("👥 고객 데이터 삽입...")

customers_data = [
    ('ALFKI', 'Alfreds Futterkiste', 'Maria Anders', 'Sales Representative', 'Obere Str. 57', 'Berlin', None, '12209', 'Germany', '030-0074321', '030-0076545'),
    ('ANATR', 'Ana Trujillo Emparedados y helados', 'Ana Trujillo', 'Owner', 'Avda. de la Constitución 2222', 'México D.F.', None, '05021', 'Mexico', '(5) 555-4729', '(5) 555-3745'),
    ('ANTON', 'Antonio Moreno Taquería', 'Antonio Moreno', 'Owner', 'Mataderos 2312', 'México D.F.', None, '05023', 'Mexico', '(5) 555-3932', None),
    ('AROUT', 'Around the Horn', 'Thomas Hardy', 'Sales Representative', '120 Hanover Sq.', 'London', None, 'WA1 1DP', 'UK', '(171) 555-7788', '(171) 555-6750'),
    ('BERGS', 'Berglunds snabbköp', 'Christina Berglund', 'Order Administrator', 'Berguvsvägen 8', 'Luleå', None, 'S-958 22', 'Sweden', '0921-12 34 65', '0921-12 34 67'),
    ('BLAUS', 'Blauer See Delikatessen', 'Hanna Moos', 'Sales Representative', 'Forsterstr. 57', 'Mannheim', None, '68306', 'Germany', '0621-08460', '0621-08924'),
    ('BLONP', 'Blondesddsl père et fils', 'Frédérique Citeaux', 'Marketing Manager', '24, place Kléber', 'Strasbourg', None, '67000', 'France', '88.60.15.31', '88.60.15.32'),
    ('BOLID', 'Bólido Comidas preparadas', 'Martín Sommer', 'Owner', 'C/ Araquil, 67', 'Madrid', None, '28023', 'Spain', '(91) 555 22 82', '(91) 555 91 99'),
    ('BONAP', 'Bon app\'', 'Laurence Lebihan', 'Owner', '12, rue des Bouchers', 'Marseille', None, '13008', 'France', '91.24.45.40', '91.24.45.41'),
    ('BOTTM', 'Bottom-Dollar Markets', 'Elizabeth Lincoln', 'Accounting Manager', '23 Tsawassen Blvd.', 'Tsawassen', 'BC', 'T2F 8M4', 'Canada', '(604) 555-4729', '(604) 555-3745')
]

cursor.executemany(
    """INSERT INTO customers (customer_id, company_name, contact_name, contact_title, 
       address, city, region, postal_code, country, phone, fax) 
       VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""",
    customers_data
)

print(f"✅ {len(customers_data)}개 고객 삽입 완료")

In [ ]:
# 8. 샘플 데이터 삽입 - Employees (직원)
print("👨‍💼 직원 데이터 삽입...")

employees_data = [
    (1, 'Davolio', 'Nancy', 'Sales Representative', 'Ms.', '1948-12-08', '1992-05-01', '507 - 20th Ave. E. Apt. 2A', 'Seattle', 'WA', '98122', 'USA', '(206) 555-9857', '5467', None, 'Education includes a BA in psychology from Colorado State University in 1970.', 2),
    (2, 'Fuller', 'Andrew', 'Vice President, Sales', 'Dr.', '1952-02-19', '1992-08-14', '908 W. Capital Way', 'Tacoma', 'WA', '98401', 'USA', '(206) 555-9482', '3457', None, 'Andrew received his BTS commercial in 1974 and a Ph.D. in international marketing from the University of Dallas in 1981.', None),
    (3, 'Leverling', 'Janet', 'Sales Representative', 'Ms.', '1963-08-30', '1992-04-01', '722 Moss Bay Blvd.', 'Kirkland', 'WA', '98033', 'USA', '(206) 555-3412', '3355', None, 'Janet has a BS degree in chemistry from Boston College (1984).', 2),
    (4, 'Peacock', 'Margaret', 'Sales Representative', 'Mrs.', '1937-09-19', '1993-05-03', '4110 Old Redmond Rd.', 'Redmond', 'WA', '98052', 'USA', '(206) 555-8122', '5176', None, 'Margaret holds a BA in English literature from Concordia College (1958) and an MA from the American Institute of Culinary Arts (1966).', 2),
    (5, 'Buchanan', 'Steven', 'Sales Manager', 'Mr.', '1955-03-04', '1993-10-17', '14 Garrett Hill', 'London', None, 'SW1 8JR', 'UK', '(71) 555-4848', '3453', None, 'Steven Buchanan graduated from St. Andrews University, Scotland, with a BSC degree in 1976.', 2)
]

cursor.executemany(
    """INSERT INTO employees (employee_id, last_name, first_name, title, title_of_courtesy, 
       birth_date, hire_date, address, city, region, postal_code, country, home_phone, 
       extension, photo, notes, reports_to) 
       VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""",
    employees_data
)

print(f"✅ {len(employees_data)}개 직원 정보 삽입 완료")

In [ ]:
# 9. 샘플 데이터 삽입 - Shippers (운송업체)
print("🚚 운송업체 데이터 삽입...")

shippers_data = [
    (1, 'Speedy Express', '(503) 555-9831'),
    (2, 'United Package', '(503) 555-3199'),
    (3, 'Federal Shipping', '(503) 555-9931')
]

cursor.executemany(
    "INSERT INTO shippers (shipper_id, company_name, phone) VALUES (?, ?, ?)",
    shippers_data
)

print(f"✅ {len(shippers_data)}개 운송업체 삽입 완료")

In [ ]:
# 10. 샘플 데이터 삽입 - Orders (주문)
print("📋 주문 데이터 삽입...")

orders_data = [
    (10248, 'ALFKI', 5, '1996-07-04', '1996-08-01', '1996-07-16', 3, 32.3800, 'Alfreds Futterkiste', 'Obere Str. 57', 'Berlin', None, '12209', 'Germany'),
    (10249, 'ANATR', 6, '1996-07-05', '1996-08-16', '1996-07-10', 1, 11.6100, 'Ana Trujillo Emparedados y helados', 'Avda. de la Constitución 2222', 'México D.F.', None, '05021', 'Mexico'),
    (10250, 'ANTON', 4, '1996-07-08', '1996-08-05', '1996-07-12', 2, 65.8300, 'Antonio Moreno Taquería', 'Mataderos 2312', 'México D.F.', None, '05023', 'Mexico'),
    (10251, 'AROUT', 3, '1996-07-08', '1996-08-05', '1996-07-15', 1, 41.3400, 'Around the Horn', '120 Hanover Sq.', 'London', None, 'WA1 1DP', 'UK'),
    (10252, 'BERGS', 4, '1996-07-09', '1996-08-06', '1996-07-11', 2, 51.3000, 'Berglunds snabbköp', 'Berguvsvägen 8', 'Luleå', None, 'S-958 22', 'Sweden'),
    (10253, 'BLAUS', 3, '1996-07-10', '1996-07-24', '1996-07-16', 2, 58.1700, 'Blauer See Delikatessen', 'Forsterstr. 57', 'Mannheim', None, '68306', 'Germany'),
    (10254, 'BLONP', 5, '1996-07-11', '1996-08-08', '1996-07-23', 2, 22.9800, 'Blondesddsl père et fils', '24, place Kléber', 'Strasbourg', None, '67000', 'France'),
    (10255, 'BOLID', 9, '1996-07-12', '1996-08-09', '1996-07-15', 3, 148.3300, 'Bólido Comidas preparadas', 'C/ Araquil, 67', 'Madrid', None, '28023', 'Spain'),
    (10256, 'BONAP', 3, '1996-07-15', '1996-08-12', '1996-07-17', 2, 13.9700, 'Bon app\'', '12, rue des Bouchers', 'Marseille', None, '13008', 'France'),
    (10257, 'BOTTM', 4, '1996-07-16', '1996-08-13', '1996-07-22', 3, 81.9100, 'Bottom-Dollar Markets', '23 Tsawassen Blvd.', 'Tsawassen', 'BC', 'T2F 8M4', 'Canada')
]

cursor.executemany(
    """INSERT INTO orders (order_id, customer_id, employee_id, order_date, required_date, 
       shipped_date, ship_via, freight, ship_name, ship_address, ship_city, ship_region, 
       ship_postal_code, ship_country) 
       VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""",
    orders_data
)

print(f"✅ {len(orders_data)}개 주문 삽입 완료")

In [ ]:
# 11. 샘플 데이터 삽입 - Order Details (주문 상세)
print("📦 주문 상세 데이터 삽입...")

order_details_data = [
    (10248, 11, 14.0000, 12, 0.0),
    (10248, 42, 9.8000, 10, 0.0),
    (10248, 72, 34.8000, 5, 0.0),
    (10249, 14, 18.6000, 9, 0.0),
    (10249, 51, 42.4000, 40, 0.0),
    (10250, 41, 7.7000, 10, 0.0),
    (10250, 51, 42.4000, 35, 0.15),
    (10250, 65, 16.8000, 15, 0.15),
    (10251, 22, 16.8000, 6, 0.05),
    (10251, 57, 15.6000, 15, 0.05),
    (10252, 20, 64.8000, 40, 0.05),
    (10252, 33, 2.0000, 25, 0.05),
    (10252, 60, 27.2000, 40, 0.0),
    (10253, 31, 10.0000, 20, 0.0),
    (10253, 39, 14.4000, 42, 0.0),
    (10253, 49, 16.0000, 40, 0.0),
    (10254, 24, 3.6000, 15, 0.15),
    (10254, 55, 19.2000, 21, 0.15),
    (10254, 74, 8.0000, 21, 0.0),
    (10255, 2, 15.2000, 20, 0.0)
]

cursor.executemany(
    "INSERT INTO order_details (order_id, product_id, unit_price, quantity, discount) VALUES (?, ?, ?, ?, ?)",
    order_details_data
)

print(f"✅ {len(order_details_data)}개 주문 상세 삽입 완료")

# 모든 변경사항 커밋
conn.commit()
print("\n💾 모든 데이터 변경사항 저장 완료!")

In [ ]:
# 12. 데이터베이스 검증 및 통계
print("📊 데이터베이스 검증 및 통계...")
print("=" * 50)

# 각 테이블별 레코드 수 확인
tables = ['categories', 'suppliers', 'products', 'customers', 'employees', 'shippers', 'orders', 'order_details']

total_records = 0
for table in tables:
    cursor.execute(f"SELECT COUNT(*) FROM {table}")
    count = cursor.fetchone()[0]
    total_records += count
    print(f"📋 {table:15}: {count:6,}개 레코드")

print(f"\n📊 총 레코드 수: {total_records:,}개")

# 데이터베이스 파일 크기
file_size = db_path.stat().st_size
print(f"💾 데이터베이스 크기: {file_size:,} bytes ({file_size/1024:.1f} KB)")

print("\n✅ Northwind 데이터베이스 구축 완료!")

In [ ]:
# 13. Text-to-SQL 테스트 쿼리 실행
print("🧪 Text-to-SQL 샘플 쿼리 테스트...")
print("=" * 50)

# 테스트 쿼리들
test_queries = [
    {
        'description': '전체 카테고리 목록',
        'sql': 'SELECT category_id, category_name, description FROM categories ORDER BY category_id'
    },
    {
        'description': '가장 비싼 상품 TOP 5',
        'sql': '''SELECT p.product_name, p.unit_price, c.category_name, s.company_name as supplier
                  FROM products p
                  JOIN categories c ON p.category_id = c.category_id
                  JOIN suppliers s ON p.supplier_id = s.supplier_id
                  ORDER BY p.unit_price DESC
                  LIMIT 5'''
    },
    {
        'description': '카테고리별 상품 수',
        'sql': '''SELECT c.category_name, COUNT(p.product_id) as product_count
                  FROM categories c
                  LEFT JOIN products p ON c.category_id = p.category_id
                  GROUP BY c.category_id, c.category_name
                  ORDER BY product_count DESC'''
    },
    {
        'description': '월별 주문 통계',
        'sql': '''SELECT strftime('%Y-%m', order_date) as order_month, 
                         COUNT(*) as order_count,
                         ROUND(AVG(freight), 2) as avg_freight
                  FROM orders 
                  WHERE order_date IS NOT NULL
                  GROUP BY strftime('%Y-%m', order_date)
                  ORDER BY order_month'''
    },
    {
        'description': '고객별 총 주문 금액',
        'sql': '''SELECT c.company_name, c.country,
                         COUNT(o.order_id) as order_count,
                         ROUND(SUM(od.unit_price * od.quantity * (1 - od.discount)), 2) as total_amount
                  FROM customers c
                  JOIN orders o ON c.customer_id = o.customer_id
                  JOIN order_details od ON o.order_id = od.order_id
                  GROUP BY c.customer_id, c.company_name, c.country
                  ORDER BY total_amount DESC
                  LIMIT 5'''
    }
]

# 쿼리 실행 및 결과 출력
for i, query in enumerate(test_queries, 1):
    print(f"\n{i}. {query['description']}:")
    print(f"SQL: {query['sql']}")
    
    try:
        cursor.execute(query['sql'])
        results = cursor.fetchall()
        
        if results:
            # 컬럼명 가져오기
            columns = [description[0] for description in cursor.description]
            
            # pandas DataFrame으로 표시
            df = pd.DataFrame(results, columns=columns)
            print("결과:")
            print(df.to_string(index=False))
        else:
            print("결과: 데이터 없음")
            
    except Exception as e:
        print(f"❌ 쿼리 실행 오류: {str(e)}")
    
    print("-" * 50)

print("\n🎉 Text-to-SQL 테스트 완료!")
print("▶️ 이제 LangChain Agent에서 이 데이터베이스를 사용할 수 있습니다.")

In [ ]:
# 14. 연결 정보 및 사용법 안내
print("📋 데이터베이스 연결 정보")
print("=" * 50)

print(f"📂 데이터베이스 파일: {db_path.absolute()}")
print(f"🔗 연결 문자열: sqlite:///{db_path.absolute()}")
print(f"📊 총 테이블: {len(tables)}개")
print(f"📋 총 레코드: {total_records:,}개")

print("\n🔧 Python에서 연결하는 방법:")
print(f"```python")
print(f"import sqlite3")
print(f"conn = sqlite3.connect('{db_path}')")
print(f"cursor = conn.cursor()")
print(f"```")

print("\n🔧 SQLAlchemy로 연결하는 방법:")
print(f"```python")
print(f"from sqlalchemy import create_engine")
print(f"engine = create_engine('sqlite:///{db_path}')")
print(f"```")

print("\n🔧 pandas로 연결하는 방법:")
print(f"```python")
print(f"import pandas as pd")
print(f"import sqlite3")
print(f"conn = sqlite3.connect('{db_path}')")
print(f"df = pd.read_sql_query('SELECT * FROM products LIMIT 5', conn)")
print(f"```")

print("\n📚 주요 테이블 관계:")
print("   - products ← categories (category_id)")
print("   - products ← suppliers (supplier_id)")
print("   - orders ← customers (customer_id)")
print("   - orders ← employees (employee_id)")
print("   - orders ← shippers (ship_via)")
print("   - order_details ← orders (order_id)")
print("   - order_details ← products (product_id)")

print("\n🎯 Text-to-SQL 샘플 질문들:")
sample_questions = [
    "가장 비싼 상품 5개를 보여주세요",
    "각 카테고리별로 상품이 몇 개씩 있나요?",
    "독일 고객들이 주문한 총 금액은 얼마인가요?",
    "1996년 7월에 몇 개의 주문이 있었나요?",
    "재고가 가장 많은 상품은 무엇인가요?",
    "직원별로 처리한 주문 수를 보여주세요",
    "운송비가 가장 비싼 주문 10개를 찾아주세요",
    "할인이 적용된 주문 상세를 모두 보여주세요"
]

for i, question in enumerate(sample_questions, 1):
    print(f"   {i}. {question}")

# 연결 종료
conn.close()
print(f"\n🔒 데이터베이스 연결 종료")
print(f"✅ 로컬 Northwind 데이터베이스 구축 완료!")
print(f"📅 완료 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")